In [401]:
import networkx as nx
import random

In [25]:
class CPM(nx.DiGraph):

    def __init__(self):
        super().__init__()
        self._dirty = True
        self._makespan = -1
        self._criticalPath = None

    def add_node(self, *args, **kwargs):
        self._dirty = True
        super().add_node(*args, **kwargs)

    def add_nodes_from(self, *args, **kwargs):
        self._dirty = True
        super().add_nodes_from(*args, **kwargs)

    def add_edge(self, *args):  # , **kwargs):
        self._dirty = True
        super().add_edge(*args)  # , **kwargs)

    def add_edges_from(self, *args, **kwargs):
        self._dirty = True
        super().add_edges_from(*args, **kwargs)

    def remove_node(self, *args, **kwargs):
        self._dirty = True
        super().remove_node(*args, **kwargs)

    def remove_nodes_from(self, *args, **kwargs):
        self._dirty = True
        super().remove_nodes_from(*args, **kwargs)

    def remove_edge(self, *args):  # , **kwargs):
        self._dirty = True
        super().remove_edge(*args)  # , **kwargs)

    def remove_edges_from(self, *args, **kwargs):
        self._dirty = True
        super().remove_edges_from(*args, **kwargs)

    def _forward(self):
        for n in nx.topological_sort(self):
            S = max([self.node[j]['C']
                     for j in self.predecessors(n)], default=0)
            self.add_node(n, S=S, C=S + self.node[n]['p'])

    def _backward(self):
        for n in nx.topological_sort(self, reverse=True):
            Cp = min([self.node[j]['Sp']
                      for j in self.successors(n)], default=self._makespan)
            self.add_node(n, Sp=Cp - self.node[n]['p'], Cp=Cp)

    def _computeCriticalPath(self):
        G = set()
        for n in self:
            if self.node[n]['C'] == self.node[n]['Cp']:
                G.add(n)
        self._criticalPath = self.subgraph(G)

    @property
    def makespan(self):
        if self._dirty:
            self._update()
        return self._makespan

    @property
    def criticalPath(self):
        if self._dirty:
            self._update()
        return self._criticalPath

    def _update(self):
        self._forward()
        self._makespan = max(nx.get_node_attributes(self, 'C').values())
        self._backward()
        self._computeCriticalPath()
        self._dirty = False

In [512]:
def generate_nodes(processes):
    graph_nodes  = CPM()
    for process in processes:
        graph_nodes.add_node(process[0], p=process[1])
    return graph_nodes

In [513]:
def generate_edges(graph_nodes, no_processes, no_requests):
    # obtenemos lista en orden de los nodos
    nodes = graph_nodes.nodes()
    # creamos lista de listas vacias para organizar el orden de los procesos en comun
    node_list = [[] for _ in range(no_processes)]
    # separamos los nodos de procesos que pertenecen a cada pedido 
    adjacency_list = [nodes[i:i + no_processes] for i in range(0, len(nodes), no_processes)]
    # creamos los arcos para unir los nodos en el orden establecido por el proceso
    for lst in adjacency_list:
        for i in range(len(lst)-1):
            graph_nodes.add_edges_from([(lst[i], lst[i+1])])
    # agrupamos los nodos que pertenecen a la misma operacion para asignarlas al azar
    nodes = graph_nodes.nodes()
    for i in range(len(node_list)):
        for j in range(i, len(nodes), no_processes):
            node_list[i].append(nodes[j])
    # asignar el orden de uso de cada estacion aleatoriamente
    for common_processes in node_list:
        random.shuffle(common_processes)
        for i in range(len(common_processes)-1):
            graph_nodes.add_edges_from([(common_processes[i], common_processes[i+1])])
    # regresamos el grafo con precedencias
    return graph_nodes, node_list


In [535]:
requests = [(1,50),(2,3),(3,41),(4,94),(5,15),(6,13),(7,34),(8,29),(9,55),(10,73),(11,14),(12,91)]
requests2 = [(1,5),(2,3),(3,4),(4,9)]
no_processes = 4
x = generate_nodes(requests)
y, node_list = generate_edges(x,no_processes, int(len(requests)/no_processes))

In [536]:
print(y.makespan)
print(y.edges())

424
[(1, 2), (2, 10), (2, 3), (3, 4), (3, 7), (5, 9), (5, 6), (6, 2), (6, 7), (7, 8), (8, 4), (9, 1), (9, 10), (10, 11), (11, 3), (11, 12), (12, 8)]


In [525]:
def perturbate_solution(graph, node_list):
    # seleccionamos un proceso aleatorio para modificar su orden
    selected_process = random.randint(0, len(node_list)-1)
    # obtenemos los nodos involucrados en el proceso seleccionado
    nodes = node_list[selected_process]
    # creamos una lista de tuplas con esos nodos para eliminar sus conexiones
    tup_list = []
    for i in range(len(nodes)-1):
        tup_list.append((nodes[i], nodes[i+1]))
    # eliminamos las conexiones entre esos nodos
    graph.remove_edges_from(tup_list)
    # generamos un nuevo orden aleatorio
    random.shuffle(nodes)
    # creamos las conexiones nuevas en base al nuevo orden aleatorio
    for i in range(len(nodes)-1):
        graph.add_edges_from([(nodes[i], nodes[i+1])])        
    return graph, nodes

In [526]:
g, nodes = perturbate_solution(y, node_list)
g.makespan

465

In [532]:
g, nodes = perturbate_solution(g, node_list)
g.makespan

421